In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import plot_tree


suomi = pd.read_csv("suomi.csv")
X = suomi.iloc[:, :-1]
y = suomi.iloc[:, -1]

ros = RandomOverSampler(sampling_strategy='minority')
rus = RandomUnderSampler(sampling_strategy='majority')
X_resampled, y_resampled = ros.fit_resample(X, y)
X_resampled, y_resampled = rus.fit_resample(X_resampled, y_resampled)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)


skb = SelectKBest(chi2, k=10) 
X_train = skb.fit_transform(X_train, y_train)
X_test = skb.transform(X_test)


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rfc = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

print("Best parameters found by GridSearchCV:")
print(best_params)

rfc_best = RandomForestClassifier(**best_params, random_state=42)
rfc_best.fit(X_train, y_train)
y_pred = rfc_best.predict(X_test)


auc = roc_auc_score(y_test, rfc_best.predict_proba(X_test)[:, 1])
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)
accuracy = accuracy_score(y_test, y_pred)


print("Random forest model with cross-validation,feature selection and parameter tuning:")
print("Area under the curve (AUC):", auc)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Positive predictive value (PPV):", ppv)
print("Negative predictive value (NPV):", npv)  # Fixed the string closing issue here
print("Accuracy:", accuracy) 

# Feature importance visualization
feature_importances = rfc_best.feature_importances_
selected_features = X.columns[skb.get_support()]

# Sort features and their importances
sorted_idx = np.argsort(feature_importances)
sorted_features = selected_features[sorted_idx]
sorted_importances = feature_importances[sorted_idx]

plt.figure(figsize=(12, 6))
sns.barplot(x=sorted_features, y=sorted_importances, palette='viridis')
plt.title("Feature importances in the best Random Forest model")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.xticks(rotation=45)
plt.show()

